In [1]:
%matplotlib inline
import openpathsampling as paths
import numpy as np
from toy_plot_helpers import ToyPlot
plot = ToyPlot()

## Setting up the engine

We always need to set up an MD engine which knows the details of the system we're simulating. The MD engine always takes a template snapshot as an example of what the system looks like. Importantly, the template includes a `topology`, which contains information about how to calculate the interactions.

In [2]:
import openpathsampling.toy_dynamics as toys

pes = (
    toys.toy_pes.OuterWalls([1.0, 1.0], [0.0, 0.0]) + 
    toys.toy_pes.Gaussian(2.0, [1.0, 4.0], [0.0,1.0]) + 
    toys.toy_pes.Gaussian(2.0, [25.0, 1.0], [0.0, -1.0])
)

topology=paths.ToyTopology(
    n_spatial = 2,
    masses =[1.0, 1.0],
    pes = pes
)

template = paths.Snapshot(
    coordinates=np.array([[-0.5, 0.0]]), 
    velocities=np.array([[0.0,0.0]]),
    potential_energy = 0.0, # Important to set energies dimensionless otherwise kcal/mol is default
    kinetic_energy = 0.0,
    topology=topology
)

integ = toys.toy_integrators.LangevinBAOABIntegrator(dt=0.02, temperature=0.1, gamma=2.5)

options={
    'integ' : integ,
    'n_frames_max' : 5000,
    'nsteps_per_frame' : 10
}

toy_eng = toys.toy_engine.ToyEngine(
    options=options,
    template=template
)
toy_eng.initialized = True

toy_eng.current_snapshot = template

Register this engine with the PathMovers

In [3]:
paths.PathMover.engine = toy_eng

## Loading trajectories from a previous calculation

Path sampling calculations take trajectories as input, so we need an input trajectory. We get that by running the `toy_bootstrap.ipynb`. Since that's a more advanced example, don't worry about understanding everything in there. Assuming you have an initial trajectory, all you need in order to run TIS is what's included in this notebook.


In [4]:
store_bootstrap = paths.storage.Storage('toy_bootstrap.nc', mode='r')

# note that the next three lines could be combined into one; they're separated here for educational purposes
final_sampleset = store_bootstrap.samplesets.last()
final_sample = final_sampleset.samples[-1]
traj = final_sample.trajectory

## Setting up order parameter, states, and interfaces

First we define the order parameter. We do this in two stages: first, we actually define a normal Python function which takes a `Snapshot` and return a number: in this case, the x-coordinate. Then we wrap that function into an OpenPathSampling order parameter using `paths.CV_Function`.

In [5]:
def xval(snapshot):
    """Return atom 0, coordinate 0 from the given snapshot"""
    return snapshot.xyz[0][0]
    

op_xval = paths.CV_Function(name="xval", fcn=xval)

Now we use that order parameter to define several volumes of interest. First we define the states. Note that we Python's `float("inf")` notation to create infinity (and negative infinity). The choice of where to set the state boundaries is based on the image or the PES above.

Then we set the interfaces using `LambdaVolumeSet`.

In [6]:
op_xval(store_bootstrap.snapshots[0:2])

[-0.5, -0.30063349]

In [7]:
op_xval(store_bootstrap.trajectories[10])

[-0.30063349,
 -0.29671022,
 -0.29447439,
 -0.2939457,
 -0.29403764,
 -0.29353917,
 -0.29359096,
 -0.29579008,
 -0.29841143,
 -0.301034]

In [8]:
stateA = paths.LambdaVolume(op_xval, float("-inf"), -0.3)
stateB = paths.LambdaVolume(op_xval, 0.3, float("inf"))

interfaces = paths.VolumeFactory.LambdaVolumeSet(
    op_xval, 
    minvals=float("-inf"), 
    maxvals=[-0.30, -0.25, -0.20, -0.13, -0.05, 0.0]
)

## Equilibration in the path ensemble

At the end of the bootstrapping process, we have a set of paths which we can use to start TIS sampling. Before doing a production run, we should let these paths equilibrate within the ensembles. If you generate your initial path through dynamics that aren't the true dynamics (e.g., using high temperature runs or using metadynamics), this stage is **extremely** important. It's important here as well, but less so than if your initial path is non-physical.

First, we'll set up the `TISTransition` we're interested in. Since this particular calculation is just studying the A-to-B transition, we use this object. In general, you'll want to use one of the `TransitionNetwork` objects. See the `toy_network.ipynb` notebook for more information on studying networks of transitions.

In [9]:
tis_trans = paths.TISTransition(
    stateA=stateA,
    stateB=stateB,
    interfaces=interfaces,
    name="A->B equil",
    orderparameter=None # default order parameter for analysis; not used
)

In [10]:
initial_globalstate = paths.SampleSet.map_trajectory_to_ensembles(traj, tis_trans.ensembles)

In [11]:
initial_globalstate.sanity_check()

This will be an example of a "normal" TIS run, i.e., TIS without replica exchange. We will sampling using only shooter movers (the same ones defined for the bootstrapping) and path reversal movers.

In [12]:
try:
    store_equil = paths.storage.Storage("toy_equil.nc", "w", template=template)
except RuntimeError:
    pass
equilibration = paths.PathSampling(
    storage=store_equil, 
    engine=toy_eng, 
    root_mover=tis_trans.default_movers(toy_eng),
    globalstate=initial_globalstate
)
equilibration.save_frequency=5

In [13]:
equilibration.globalstate.sanity_check()

In [ ]:
%%prun -s cumulative
while not stateA(equilibration.globalstate[0].trajectory[-1]): # keep equilibrating until innermost can extend to minus CLEAN THIS LATER
    print "Still not equilibrated..."
    equilibration.run(10)

Still not equilibrated...
[None]
[0.31872344]
[0.31872344]
[0.31872344]
[None]
[0.29723588]
[0.29723588]
[0.29723588]
[0.31872344]
[0.31872344]
[0.29723588]
[0.29723588]
[None]
[0.27663738]
[None]
[0.25678003]
[None]
[0.23800957]
[None]
[0.21999347]
[None]
[None]
[None]
[None]
[0.31872344]
[0.31872344]
[0.31872344]
[0.31872344]
[0.29723588]
[0.29723588]
[0.29723588]
[0.29723588]
[0.31872344]
[0.31872344]
[0.29723588]
[0.29723588]
[0.27663738]
[0.27663738]
[0.25678003]
[0.25678003]
[0.23800957]
[0.23800957]
[0.21999347]
[0.21999347]
[None]
[None]
[None]
[None]
[None]
[None]
[0.31872344]
[0.31872344]
[0.31872344]
[0.31872344]
[0.29723588]
[0.29723588]
[0.29723588]
[0.29723588]
[0.31872344]
[0.31872344]
[0.29723588]
[0.29723588]
[0.27663738]
[0.27663738]
[0.25678003]
[0.25678003]
[0.23800957]
[0.23800957]
[0.21999347]
[0.21999347]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[0.31872344]
[0.31872344]
[0.31872344]
[0.31872344]
[0.29723588]
[0.29723588]
[0.29723588]
[0.29723588]


In [ ]:
equilibration.globalstate.sanity_check()

In [ ]:
list(list.__iter__(store_equil.snapshots.all()[2:3]))

## RETIS: Production run

Now we'll do a full production run using replica exchange TIS (RETIS).

First, we create a `RETISTransition`, in the same way that we created the `TISTransition`.

In [17]:
retis_trans = paths.RETISTransition(
    stateA=stateA,
    stateB=stateB,
    interfaces=interfaces,
    name="A->B",
    orderparameter=op_xval # we'll use this in the analysis notebook
)
start_gs = paths.SampleSet.translate_ensembles(equilibration.globalstate, retis_trans.ensembles)

{Storage @ 'toy_equil.nc': [-0.30915508]}
[-0.30915508]
{Storage @ 'toy_equil.nc': [-0.30915508]}
[-0.30915508]
{Storage @ 'toy_equil.nc': [-0.29382232]}
[-0.29382232]
{Storage @ 'toy_equil.nc': [-0.29382232]}
[-0.29382232]
{Storage @ 'toy_equil.nc': [-0.29382232]}
[-0.29382232]
{Storage @ 'toy_equil.nc': [-0.29382232]}
[-0.29382232]
{Storage @ 'toy_equil.nc': [-0.27940929]}
[-0.27940929]
{Storage @ 'toy_equil.nc': [-0.27940929]}
[-0.27940929]
{Storage @ 'toy_equil.nc': [-0.26524594]}
[-0.26524594]
{Storage @ 'toy_equil.nc': [-0.26524594]}
[-0.26524594]
{Storage @ 'toy_equil.nc': [-0.25150433]}
[-0.25150433]
{Storage @ 'toy_equil.nc': [-0.25150433]}
[-0.25150433]
{Storage @ 'toy_equil.nc': [-0.23896274]}
[-0.23896274]
{Storage @ 'toy_equil.nc': [-0.23896274]}
[-0.23896274]
{Storage @ 'toy_equil.nc': [-0.2261869]}
[-0.2261869]
{Storage @ 'toy_equil.nc': [-0.2261869]}
[-0.2261869]
{Storage @ 'toy_equil.nc': [-0.21332669]}
[-0.21332669]
{Storage @ 'toy_equil.nc': [-0.21332669]}
[-0.213326

At this point, we still don't have a path that satisfies the minus interface. 

In [18]:
minus_segment = equilibration.globalstate[0].trajectory
last_frame = minus_segment[-1]
extension = toy_eng.generate(last_frame, [retis_trans.minus_ensemble.can_append])
first_minus = minus_segment + extension[1:]
minus_samp = paths.Sample(
    replica=-1,
    trajectory=first_minus,
    ensemble=retis_trans.minus_ensemble
)
start_gs = start_gs.apply_samples([minus_samp])

{Storage @ 'toy_equil.nc': [-0.30631357]}
[-0.30631357]
{Storage @ 'toy_equil.nc': [-0.30631357]}
[-0.30631357]
{Storage @ 'toy_equil.nc': [-0.30631357]}
[-0.30631357]
{Storage @ 'toy_equil.nc': [-0.30631357]}
[-0.30631357]
{Storage @ 'toy_equil.nc': [-0.30631357]}
[-0.30631357]
{Storage @ 'toy_equil.nc': [-0.30631357]}
[-0.30631357]
{Storage @ 'toy_equil.nc': [None]}
[None]
{Storage @ 'toy_equil.nc': [None]}
[None]
{Storage @ 'toy_equil.nc': [None]}
[None]
{Storage @ 'toy_equil.nc': [None]}
[None]
{Storage @ 'toy_equil.nc': [None]}
[None]
{Storage @ 'toy_equil.nc': [None]}
[None]
{Storage @ 'toy_equil.nc': [None]}
[None]
{Storage @ 'toy_equil.nc': [None]}
[None]
{Storage @ 'toy_equil.nc': [None]}
[None]
{Storage @ 'toy_equil.nc': [None]}
[None]
{Storage @ 'toy_equil.nc': [None]}
[None]
{Storage @ 'toy_equil.nc': [None]}
[None]
{Storage @ 'toy_equil.nc': [None]}
[None]
{Storage @ 'toy_equil.nc': [None]}
[None]
{Storage @ 'toy_equil.nc': [-0.30631357]}
[-0.30631357]
{Storage @ 'toy_equi

In [19]:
start_gs.sanity_check()

{Storage @ 'toy_equil.nc': [-0.30915508]}
[-0.30915508]
{Storage @ 'toy_equil.nc': [-0.30915508]}
[-0.30915508]
{Storage @ 'toy_equil.nc': [-0.29382232]}
[-0.29382232]
{Storage @ 'toy_equil.nc': [-0.29382232]}
[-0.29382232]
{Storage @ 'toy_equil.nc': [-0.29382232]}
[-0.29382232]
{Storage @ 'toy_equil.nc': [-0.29382232]}
[-0.29382232]
{Storage @ 'toy_equil.nc': [-0.27940929]}
[-0.27940929]
{Storage @ 'toy_equil.nc': [-0.27940929]}
[-0.27940929]
{Storage @ 'toy_equil.nc': [-0.26524594]}
[-0.26524594]
{Storage @ 'toy_equil.nc': [-0.26524594]}
[-0.26524594]
{Storage @ 'toy_equil.nc': [-0.25150433]}
[-0.25150433]
{Storage @ 'toy_equil.nc': [-0.25150433]}
[-0.25150433]
{Storage @ 'toy_equil.nc': [-0.23896274]}
[-0.23896274]
{Storage @ 'toy_equil.nc': [-0.23896274]}
[-0.23896274]
{Storage @ 'toy_equil.nc': [-0.2261869]}
[-0.2261869]
{Storage @ 'toy_equil.nc': [-0.2261869]}
[-0.2261869]
{Storage @ 'toy_equil.nc': [-0.21332669]}
[-0.21332669]
{Storage @ 'toy_equil.nc': [-0.21332669]}
[-0.213326

In [20]:
import logging.config
logging.config.fileConfig("logging.conf", disable_existing_loggers=False)

In [21]:
try:
    store_retis = paths.storage.Storage("toy_retis.nc", "w", template=template)
except RuntimeError:
    pass

store_retis.save(retis_trans) # TODO

True

In [22]:
production = paths.PathSampling(
    storage=store_retis,
    engine=toy_eng,
    root_mover=retis_trans.default_movers(toy_eng),
    globalstate=start_gs
)
production.save_frequency = 10

{Storage @ 'toy_retis.nc': [None], Storage @ 'toy_equil.nc': [-0.30915508]}
[None]
{Storage @ 'toy_retis.nc': [None], Storage @ 'toy_equil.nc': [-0.30915508]}
[None]
{Storage @ 'toy_retis.nc': [None], Storage @ 'toy_equil.nc': [-0.29382232]}
[None]
{Storage @ 'toy_retis.nc': [None], Storage @ 'toy_equil.nc': [-0.29382232]}
[None]
{Storage @ 'toy_retis.nc': [None], Storage @ 'toy_equil.nc': [-0.29382232]}
[None]
{Storage @ 'toy_retis.nc': [None], Storage @ 'toy_equil.nc': [-0.29382232]}
[None]
{Storage @ 'toy_retis.nc': [None], Storage @ 'toy_equil.nc': [-0.27940929]}
[None]
{Storage @ 'toy_retis.nc': [None], Storage @ 'toy_equil.nc': [-0.27940929]}
[None]
{Storage @ 'toy_retis.nc': [None], Storage @ 'toy_equil.nc': [-0.26524594]}
[None]
{Storage @ 'toy_retis.nc': [None], Storage @ 'toy_equil.nc': [-0.26524594]}
[None]
{Storage @ 'toy_retis.nc': [None], Storage @ 'toy_equil.nc': [-0.25150433]}
[None]
{Storage @ 'toy_retis.nc': [None], Storage @ 'toy_equil.nc': [-0.25150433]}
[None]
{Sto

In [23]:
store_retis.sync() # actually, this should be done in the calculation object

In [24]:
#%load_ext line_profiler

In [25]:
#%lprun  -f paths.AllInXEnsemble.can_append  \
#        -f paths.AllInXEnsemble.__call__ \
#        -f paths.SequentialEnsemble.can_append \
#        -f paths.SequentialEnsemble._find_subtraj_final \
#        -f paths.LambdaVolume.__call__ \
#        production.run(50)


In [26]:
production.run(1)

Storage @ 'toy_retis.nc' [118]
{Storage @ 'toy_retis.nc': [None], Storage @ 'toy_equil.nc': [0.30511573]}
[None]
{Storage @ 'toy_retis.nc': [0.30511573], Storage @ 'toy_equil.nc': [0.30511573]}
[[0.30511573]]
{Storage @ 'toy_retis.nc': [0.30511573], Storage @ 'toy_equil.nc': [0.30511573]}
[[0.30511573]]
{Storage @ 'toy_retis.nc': [0.30511573], Storage @ 'toy_equil.nc': [0.30511573]}
[[0.30511573]]
{Storage @ 'toy_retis.nc': [0.30511573], Storage @ 'toy_equil.nc': [0.30511573]}
[[0.30511573]]
{Storage @ 'toy_retis.nc': [0.30511573], Storage @ 'toy_equil.nc': [0.30511573]}
[[0.30511573]]
Storage @ 'toy_retis.nc' [117]
{Storage @ 'toy_retis.nc': [None], Storage @ 'toy_equil.nc': [0.28635105]}
[None]
{Storage @ 'toy_retis.nc': [0.28635105], Storage @ 'toy_equil.nc': [0.28635105]}
[[0.28635105]]
Storage @ 'toy_retis.nc' [116]
{Storage @ 'toy_retis.nc': [None], Storage @ 'toy_equil.nc': [0.26753804]}
[None]
{Storage @ 'toy_retis.nc': [0.26753804], Storage @ 'toy_equil.nc': [0.26753804]}
[[0.

AssertionError: 

In [27]:
print store_bootstrap.snapshots.count()

3081


In [27]:
#print list(list.__iter__(traj))
print traj[1].idx
#op_xval(traj)
print list.__getitem__(traj,1)[0]

{Storage @ 'toy_retis.nc': 2, Storage @ 'toy_equil.nc': 2, Storage @ 'toy_bootstrap.nc': 3047}
SnapshotStore(content_class=<class 'openpathsampling.snapshot.Snapshot'>, variable_prefix=snapshot)


In [28]:
len(store_retis.snapshots)

287

In [30]:
sn = store_retis.snapshots[3]
print sn
print op_xval(sn)
op_xval(traj[0:4])

{Storage @ 'toy_retis.nc': [-0.27940929], Storage @ 'toy_equil.nc': [-0.27940929]}
[[-0.27940929]]
[-0.27940929]
{Storage @ 'toy_retis.nc': [-0.27940929, -0.30915508, -0.26524594, -0.29382232], Storage @ 'toy_equil.nc': [-0.27940929, -0.30915508, -0.26524594, -0.29382232]}
[[-0.27940929, -0.30915508, -0.26524594, -0.29382232], [-0.27940929, -0.30915508, -0.26524594, -0.29382232], [-0.27940929, -0.30915508, -0.26524594, -0.29382232], [-0.27940929, -0.30915508, -0.26524594, -0.29382232]]


[[-0.27940929, -0.30915508, -0.26524594, -0.29382232],
 [-0.27940929, -0.30915508, -0.26524594, -0.29382232],
 [-0.27940929, -0.30915508, -0.26524594, -0.29382232],
 [-0.27940929, -0.30915508, -0.26524594, -0.29382232]]

In [31]:
st = store_retis
op = op_xval.post.post.post.post.cod_stores[st].post
op2 = op_xval.post.post.post.post.cod_stores[st]
op3 = op_xval.post.post.post.post
print op.__class__.__name__
print op.values()
print [ob.idx[st] if type(ob) is not tuple else (ob[0].storage, ob[1]) for ob in op.keys() ]

ChainDict
[0.20719647, -0.0021424436, 0.072916158, -0.27940929, 0.22607134, -0.30915508, 0.083516829, 0.24727398, 0.013300572, 0.095692091, 0.26753804, 0.10756681, 0.28635105, 0.025195602, 0.1184947, 0.30511573, 0.0035522433, -0.27940929, 0.0092260661, 0.12961386, 0.01678532, 0.021445099, 0.14204466, 0.029079871, 0.035716139, -0.30915508, 0.15544838, 0.052102797, 0.04371817, -0.26524594, -0.0072603486, 0.059785195, 0.18950465, 0.17126556, 0.065682009, -0.29382232]
[113, 92, 103, (Storage @ 'toy_bootstrap.nc', 3048), 114, 1, 104, 115, 95, 105, 116, 106, 117, 98, 107, 118, 93, 3, 94, 108, 96, 97, 109, 99, 100, (Storage @ 'toy_bootstrap.nc', 3046), 110, 34, 33, (Storage @ 'toy_bootstrap.nc', 3049), 91, 101, 112, 111, 102, (Storage @ 'toy_bootstrap.nc', 3047)]


In [32]:
op(traj[2:10])

Storage @ 'toy_retis.nc' [4, 5, 6, 7, 8, 9, 10]


[-0.27940929, None, None, None, None, None, None, None]

In [33]:
op2(traj[2:5])


Storage @ 'toy_retis.nc' [4, 5]


[-0.27940929, None, None]

In [34]:
op3(traj[2:10])

Storage @ 'toy_retis.nc' [4, 5, 6, 7, 8, 9, 10]
{Storage @ 'toy_retis.nc': [-0.27940929, None, None, None, None, None, None, None], Storage @ 'toy_equil.nc': [-0.27940929, -0.26524594, -0.25150433, -0.23896274, -0.2261869, -0.21332669, -0.20008993, -0.18586822]}
[[-0.27940929, None, None, None, None, None, None, None], None, None, None, None, None, None, None]


[[-0.27940929, None, None, None, None, None, None, None],
 -0.26524594,
 -0.25150433,
 -0.23896274,
 -0.2261869,
 -0.21332669,
 -0.20008993,
 -0.18586822]